<a href="https://colab.research.google.com/github/okana2ki/gai4e/blob/main/emotion_sns2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 棒グラフと投稿一覧表示の英語部分を日本語に変更（Geminiに指示）

In [4]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# --- データとキュー ---
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment"])
post_queue = queue.Queue()
latest_ai_comment = ""

# --- 可視化関数 ---
def update_visualizations():
    global df
    # 感情ラベルの日本語マッピング
    sentiment_labels_jp = {
        "interest": "興味",
        "understanding": "理解",
        "confusion": "困惑",
        "neutral": "中立",
        "pending": "処理中"
    }
    df_display = df.copy()
    df_display['sentiment'] = df_display['sentiment'].map(sentiment_labels_jp)

    fig = px.histogram(
        df_display,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["興味","理解","困惑","中立","処理中"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"興味":"gold","理解":"lightblue","困惑":"orange","中立":"lightgrey","処理中":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image, df_display

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、この大学（宮崎産業経営大学）に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["interest","understanding","confusion","neutral"]:
            sentiment = "neutral"
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限対応

threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending",""]
    post_queue.put((text,row_index))
    fig, wc_image, df_display = update_visualizations()
    return fig, wc_image, "", "投稿完了。LLM解析は順番に処理されます。", df_display

# --- 更新ボタン処理 ---
def update_display(input_text_value):
    fig, wc_image, df_display = update_visualizations()
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
    else:
        ai_comment = ""
    return fig, wc_image, ai_comment, df_display

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")
    dataframe_output = gr.DataFrame(label="投稿一覧")

    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output, dataframe_output])
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, dataframe_output])

demo.launch()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e983fa13f4c2b211c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 投稿された感想を表示する機能を追加する（Geminiに指示）

In [3]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# --- データとキュー ---
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment"])
post_queue = queue.Queue()
latest_ai_comment = ""

# --- 可視化関数 ---
def update_visualizations():
    global df
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["interest","understanding","confusion","neutral","pending"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"interest":"gold","understanding":"lightblue","confusion":"orange","neutral":"lightgrey","pending":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、この大学（宮崎産業経営大学）に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["interest","understanding","confusion","neutral"]:
            sentiment = "neutral"
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限対応

threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending",""]
    post_queue.put((text,row_index))
    fig, wc_image = update_visualizations()
    return fig, wc_image, "", "投稿完了。LLM解析は順番に処理されます。", df

# --- 更新ボタン処理 ---
def update_display(input_text_value):
    fig, wc_image = update_visualizations()
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
    else:
        ai_comment = ""
    return fig, wc_image, ai_comment, df

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")
    dataframe_output = gr.DataFrame(label="投稿一覧")

    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output, dataframe_output])
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, dataframe_output])

demo.launch()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c2f7d8ca41496d198d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# システム指示の修正（Geminiに指示）

システム指示を以下のように変えて下さい。分析する感情の種類を変えたので、スクリプトの他の対応する部分もそれに応じて変えて下さい。必要最小限の修正にとどめて、他の部分はそのままにして下さい。
```
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、この大学（宮崎産業経営大学）に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
```



In [ ]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# --- データとキュー ---
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment"])
post_queue = queue.Queue()
latest_ai_comment = ""

# --- 可視化関数 ---
def update_visualizations():
    global df
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["interest","understanding","confusion","neutral","pending"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"interest":"gold","understanding":"lightblue","confusion":"orange","neutral":"lightgrey","pending":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、この大学（宮崎産業経営大学）に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["interest","understanding","confusion","neutral"]:
            sentiment = "neutral"
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限対応

threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending",""]
    post_queue.put((text,row_index))
    fig, wc_image = update_visualizations()
    return fig, wc_image, "", "投稿完了。LLM解析は順番に処理されます。"

# --- 更新ボタン処理 ---
def update_display(input_text_value):
    fig, wc_image = update_visualizations()
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
    else:
        ai_comment = ""
    return fig, wc_image, ai_comment

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output])
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output])

demo.launch()

# 以下は古いバージョン（GPT-5に依頼して作成）

In [ ]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# 投稿保存
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment"])
# LLM用キュー
post_queue = queue.Queue()
# 最新AIコメント保持（バックグラウンド用）
latest_ai_comment = ""

# --- 集計更新 ---
def update_visualizations():
    global df
    # 棒グラフ
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["joy","surprise","confusion","neutral","pending"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"joy":"gold","surprise":"lightblue","confusion":"orange","neutral":"lightgrey","pending":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    # ワードクラウド
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        # Gemini呼び出し
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : joy, surprise, confusion, neutral のいずれか"
            " 2) 'ai_comment' : 短いポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        # JSON抽出
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["joy","surprise","confusion","neutral"]:
            sentiment = "neutral"
        # DataFrame更新
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限に合わせて4秒に1回

# バックグラウンドスレッド開始
threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending",""]
    post_queue.put((text,row_index))
    fig, wc_image = update_visualizations()
    # ステータス用欄に表示
    return fig, wc_image, "", "投稿完了。LLM解析は順番に処理されます。"

# --- 更新ボタン処理（修正版） ---
def update_display(input_text_value):
    fig, wc_image = update_visualizations()

    # 感想入力欄に書かれている内容に対応するAIコメントを取得
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        # 最新の行を取得
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
    else:
        ai_comment = ""  # 投稿されていない場合は空欄

    return fig, wc_image, ai_comment

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    # 投稿ボタン：集計グラフとワードクラウド更新 + 投稿ステータス表示
    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output])
    # 更新ボタン：集計グラフ・ワードクラウド・入力欄に対応する最新AIコメント表示
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output])

demo.launch()


In [ ]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk
import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# 投稿保存
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment"])
# LLM用キュー
post_queue = queue.Queue()
# 最新AIコメント保持
latest_ai_comment = ""

# --- 集計更新 ---
def update_visualizations():
    global df
    # 棒グラフ
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["joy","surprise","confusion","neutral","pending"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"joy":"gold","surprise":"lightblue","confusion":"orange","neutral":"lightgrey","pending":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    # ワードクラウド
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        # Gemini呼び出し
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : joy, surprise, confusion, neutral のいずれか"
            " 2) 'ai_comment' : 短いポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        # JSON抽出
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["joy","surprise","confusion","neutral"]:
            sentiment = "neutral"
        # DataFrame更新
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限に合わせて4秒に1回

# バックグラウンドスレッド開始
threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending",""]
    post_queue.put((text,row_index))
    fig, wc_image = update_visualizations()
    return fig, wc_image, "", "投稿完了。LLM解析は順番に処理されます。"

# --- 更新ボタン処理 ---
def update_display():
    fig, wc_image = update_visualizations()
    return fig, wc_image, latest_ai_comment

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    # 投稿ボタン：集計グラフとワードクラウド更新 + 投稿ステータス表示
    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output])
    # 更新ボタン：集計グラフ・ワードクラウド・最新AIコメント表示
    update_btn.click(update_display, outputs=[graph_output, wc_output, ai_comment_output])

demo.launch()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://805bf316c755db9500.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk
import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# 投稿保存
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment"])
# LLM用キュー
post_queue = queue.Queue()

# --- 集計更新 ---
def update_visualizations():
    global df
    # 棒グラフ
    fig = px.histogram(
        df,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["joy","surprise","confusion","neutral"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"joy":"gold","surprise":"lightblue","confusion":"orange","neutral":"lightgrey"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    # ワードクラウド
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        # Gemini呼び出し
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : joy, surprise, confusion, neutral のいずれか"
            " 2) 'ai_comment' : 短いポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        # JSON抽出
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["joy","surprise","confusion","neutral"]:
            sentiment = "neutral"
        # DataFrame更新
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限に合わせて4秒に1回

# バックグラウンドスレッド開始
threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending","pending"]
    post_queue.put((text,row_index))
    fig, wc_image = update_visualizations()
    return fig, wc_image, "投稿完了。LLM解析は順番に処理されます。"

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")

    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output])
    update_btn.click(lambda: update_visualizations(), outputs=[graph_output, wc_output])

demo.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
Reading package lists... Done
Building dependency tree..